# Here we try to implement Fairness through Awareness by Dwork et. al.

## Since the original formulation has $O(n^2)$ constraints we try a heuristic approach

Import necessary dependencies.
Note that the modules "utils_modified" and "funcs_disp_mist_modified" containing some helper functions are adopted from "Fairness Beyond Disparate Treatment & Disparate Impact: Learning Classification without Disparate Mistreatment" by Zafar et. al. (https://github.com/mbilalzafar/fair-classification)

In [1]:
import scipy.io
import sys
import numpy as np
import compas_preprocessing as preproc
import funcs_disp_mist_modified as fdm
from helpers import *
from random import shuffle
from cvxpy import *
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit

## Importing the COMPAS dataset

Here we import the COMPAS dataset. It is preprocessed the same way as in the ProPublica study (refer to https://github.com/propublica/compas-analysis)

In [2]:
X, y, x_control = preproc.load_compas_data_modified()
sensitive_attrs = list(x_control.keys())

# since we want +1 to be the desired outcome we have to flip our training labels
# previously +1 indicated that a person recidivated within 2 years
y = -y

Looking for file 'compas-scores-two-years.csv' in the current directory...
File found in current directory..

Number of people recidivating within two years
-1    2795
 1    2483
dtype: int64


Features we will be using for classification are: ['intercept', 'age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'race', 'sex', 'priors_count', 'c_charge_degree'] 



Here we compute the maximum distance between any two data points in order to be able to compute the normalized distance

In [3]:
n = X.shape[0]
D = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        D[i,j] = np.linalg.norm(X[i,:]- X[j,:])
d_max = D.max()

Setting some parameters for the solver

In [4]:
max_iters = 100 # for the convex program
max_iter_dccp = 50  # for the dccp algo
tau, mu = 0.005, 1.2 # default dccp parameters, need to be varied per dataset

## Approach

We will first train an unconstrained classifier on the training set and evaluate which constraints are violated by more than delta. A priori we determine how many constraints we can enforce with reasonable computational effort (max_num_cons). Then we choose delta as large as possible such that we still have more violations that we can enforce. That is done in an approximate fashion because it stays the same across all train/test splits.
We generate a list of all those violations. Then we iterate over and enforce x constraints where 0<=x<=max_num_cons. We evaluate on the validation set how many constraints are still violated and the average violation of the constraints.

### Generate cross validation splits for both training and evaulation

In [5]:
cross_val_iter = 10
kf = KFold(n_splits=cross_val_iter,shuffle=True)

## Here we train our classifier with different alpha and tau values and save the resulting weight vectors

Note that you need to run this part first before running the evaluation part. Also if the solver fails in one cross validation iteration the value of the weight vector is discarded. However, the computation for that only works after the first cross validation iteration. So if the solver does fail in the first iteration, you get a corresponding error message and need to rerun the training part.

In [6]:
max_num_cons = 9000
iterations = 20
delta = 0.55
num_cons_array = np.linspace(0,max_num_cons,iterations)

# number of features in the dataset
no_features = X.shape[1]

# Here we will put the weights resulting from the training of the classifier
weight_matrix = np.zeros((iterations,no_features))

i = 0
for train_index, test_index in kf.split(X):
    print("#############cross validation iteration " + str(i) + " #########################")
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_control_train = {'race':x_control['race'][train_index]}
    x_control_test = {'race':x_control['race'][test_index]}
    num_points, num_features = x_train.shape
    
    # initialize weight vectors to a random value
    w_uncons = Variable(num_features)
    w_uncons.value = np.random.rand(num_features)
    # logistic loss
    loss = sum_entries(logistic( mul_elemwise(-y_train, x_train*w_uncons) )  ) / num_points # we are converting y to a diagonal matrix for consistent

    prob = Problem(Minimize(loss),[norm(w_uncons) == 1])

    try:
        prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
                solver=ECOS, verbose=False,
                max_iters=max_iters, max_iter=max_iter_dccp)
    except Exception as e:
        print (e)

    #convert solution vector to a numpy array
    w_uncons = np.array(w_uncons.value).flatten()
    
    # list of all constraints that are violated by an unconstrained classifier
    vio_list_uncons = violated_pairs(np.reciprocal(1+np.exp(-np.dot(x_train, w_uncons))), x_train, delta, d_max)
    print("we could enforce up to this many violations:")
    print(len(vio_list_uncons))
    
    shuffle(vio_list_uncons)
    for k in range(iterations):
        print("================= iteration: " + str(k) + " ====================")

        # initialize weight vectors to a random value
        w_cons = Variable(num_features)
        w_cons.value = np.random.rand(x_train.shape[1])

        # logistic loss
        loss = sum_entries(logistic( mul_elemwise(-y_train, x_train*w_cons) )  ) / num_points # we are converting y to a diagonal matrix for consistent

        #scale the predictions to be in [-1,1]
        y_pred = (x_train*w_cons)/5

        constraints = [norm(w_cons) == 1]

        num_cons = int(num_cons_array[k])
        
        # now we add the obtained constraints to our optimization problem
        for (i1,j1) in vio_list_uncons[:num_cons]:
            constraints.append(abs(y_pred[i1] - y_pred[j1]) <= distance(x_train[i1,:],x_train[j1,:], d_max))


        prob = Problem(Minimize(loss),constraints)

        solver_failed = False
        
        try:
            prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
                    solver=ECOS, verbose=False,
                    max_iters=max_iters, max_iter=max_iter_dccp)
        except Exception as e:
            print (e)
            solver_failed = True
            
        if prob.status != "Converged" and prob.status != "OPTIMAL":
                solver_failed = True

        if not solver_failed: 
            w_cons = np.array(w_cons.value).flatten()
            
            print(w_cons)
            
            weight_matrix[k,:] += w_cons/cross_val_iter
            
            train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_cons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)
        else:
            # test if all values 0, which they are only in the first cross validation iteration
            if not np.any(weight_matrix[k,:]):
                sys.exit("Solver failed in first cross validation iteration")
            else:
                weight_matrix[k,:] = weight_matrix[k,:]*(i + 1)/i

        print(prob.status)
    i += 1

#############cross validation iteration 0 #########################
we could enforce up to this many violations:
102578
================= iteration: 0 ====================
[ 0.20900779  0.50918733 -0.0252926   0.00976185 -0.12842913  0.06675674
 -0.25524398 -0.75651861  0.19464992]
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Converged
================= iteration: 1 ====================
[ 0.14532862  0.07531194 -0.03595068 -0.04321315 -0.0472088   0.01116118
 -0.03803255 -0.10922222  0.02559014]
[ 1.  1.  1. ... -1.  1.  1.]


Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.10 ||
||  1  || 0.87 || 0.06 ||


Converged
================= iteration: 2 ====================
[ 0.13228576  0.08774723 -0.03929965 -0.03441464 -0.04268266  0.01447607
 -0.02949419 -0.09971054  0.03499608]
[ 1.  1.  1. ... -1.  1.  1.]


Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.66 || 0.12 ||
||  1  || 0.87 



Accuracy: 0.623
||  s  || FPR. || FNR. ||
||  0  || 0.69 || 0.07 ||
||  1  || 0.88 || 0.05 ||


Converged
================= iteration: 7 ====================
[ 0.13319319  0.07256716 -0.04053846 -0.02866478 -0.04039366  0.00927293
 -0.02109355 -0.11479689  0.0204311 ]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.623
||  s  || FPR. || FNR. ||
||  0  || 0.69 || 0.07 ||
||  1  || 0.88 || 0.05 ||


Converged
================= iteration: 8 ====================
[ 0.13442618  0.07513056 -0.03521078 -0.02856518 -0.03502459  0.01223124
 -0.02254164 -0.11528437  0.01888381]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.69 || 0.08 ||
||  1  || 0.89 || 0.06 ||


Converged
================= iteration: 9 ====================
[ 0.13512237  0.07354009 -0.03571525 -0.02981814 -0.03525108  0.01196009
 -0.0236824  -0.11555035  0.01839608]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.619
||  s  || FPR. || FNR. ||
||  0  || 0.69 || 0.07 ||
||  1  || 0.89 || 0.06 ||


Converged
=======

[ 0.12604579  0.07713602 -0.03538438 -0.03731395 -0.04096246  0.02256354
 -0.01861914 -0.10903551  0.01842932]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.619
||  s  || FPR. || FNR. ||
||  0  || 0.72 || 0.10 ||
||  1  || 0.85 || 0.03 ||


Converged
================= iteration: 15 ====================
[ 0.12624299  0.07742189 -0.03550779 -0.03718312 -0.04089515  0.02169096
 -0.01918149 -0.10887558  0.01889419]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.619
||  s  || FPR. || FNR. ||
||  0  || 0.72 || 0.10 ||
||  1  || 0.85 || 0.03 ||


Converged
================= iteration: 16 ====================
[ 0.1267476   0.07742189 -0.03550779 -0.03718312 -0.04089515  0.02169096
 -0.01918149 -0.10887558  0.01889419]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.619
||  s  || FPR. || FNR. ||
||  0  || 0.72 || 0.10 ||
||  1  || 0.85 || 0.03 ||


Converged
================= iteration: 17 ====================
[ 0.1315551   0.07797236 -0.0397063  -0.03315844 -0.04174819  0.01434737
 -0.02508079 -0.10723611  0.0246058 ]
[

[ 0.15718923  0.07730053 -0.04068901 -0.03242185 -0.03972219  0.02203002
 -0.02949393 -0.10994115  0.0183667 ]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.593
||  s  || FPR. || FNR. ||
||  0  || 0.72 || 0.07 ||
||  1  || 0.86 || 0.03 ||


Converged
================= iteration: 2 ====================
[ 0.15933123  0.08054656 -0.03763263 -0.03530299 -0.03874766  0.01693012
 -0.02946157 -0.10800995  0.01980354]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.593
||  s  || FPR. || FNR. ||
||  0  || 0.72 || 0.07 ||
||  1  || 0.85 || 0.03 ||


Converged
================= iteration: 3 ====================
[ 0.158014    0.07913654 -0.03716036 -0.03436859 -0.04282695  0.01816103
 -0.02892086 -0.10670369  0.02006843]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.595
||  s  || FPR. || FNR. ||
||  0  || 0.71 || 0.07 ||
||  1  || 0.85 || 0.03 ||


Converged
================= iteration: 4 ====================
[ 0.15848125  0.07929865 -0.03737451 -0.03479076 -0.04270943  0.01607895
 -0.02761388 -0.10738792  0.01870619]
[1. 



Accuracy: 0.598
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.12 ||
||  1  || 0.87 || 0.03 ||


Converged
================= iteration: 9 ====================
[ 0.14887471  0.08133425 -0.03781767 -0.02870484 -0.03784059  0.01244687
 -0.02368806 -0.10836516  0.02188002]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.598
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.12 ||
||  1  || 0.87 || 0.03 ||


Converged
================= iteration: 10 ====================
[ 0.14846372  0.08024837 -0.03841588 -0.02935865 -0.03807399  0.01088004
 -0.0225485  -0.10905254  0.02279891]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.598
||  s  || FPR. || FNR. ||
||  0  || 0.66 || 0.12 ||
||  1  || 0.86 || 0.03 ||


Converged
================= iteration: 11 ====================
[ 0.14865068  0.08024837 -0.03841588 -0.02935865 -0.03807399  0.01088004
 -0.0225485  -0.10905254  0.02279891]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.598
||  s  || FPR. || FNR. ||
||  0  || 0.66 || 0.12 ||
||  1  || 0.86 || 0.03 ||


Converged
=====



Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.06 ||
||  1  || 0.90 || 0.03 ||


Converged
================= iteration: 17 ====================
[ 0.14982655  0.07085206 -0.04000872 -0.03481975 -0.04403921  0.00993233
 -0.02672825 -0.10994098  0.02602348]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.06 ||
||  1  || 0.90 || 0.03 ||


Converged
================= iteration: 18 ====================
[ 0.14876742  0.07211514 -0.03945002 -0.03412312 -0.04254123  0.010048
 -0.0241063  -0.11095176  0.02363144]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.06 ||
||  1  || 0.90 || 0.03 ||


Converged
================= iteration: 19 ====================
[ 0.14903419  0.07302153 -0.03778248 -0.03405107 -0.04038193  0.01013361
 -0.02369127 -0.11166443  0.02275285]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.621
||  s  || FPR. || FNR. ||
||  0  || 0.64 || 0.06 ||
||  1  || 0.90 || 0.03 ||


Converged
######



Accuracy: 0.636
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.07 ||
||  1  || 0.88 || 0.03 ||


Converged
================= iteration: 4 ====================
[ 0.14230342  0.08246243 -0.04026776 -0.02592219 -0.03784429  0.00122832
 -0.02479532 -0.11235814  0.01226573]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.636
||  s  || FPR. || FNR. ||
||  0  || 0.68 || 0.07 ||
||  1  || 0.88 || 0.02 ||


Converged
================= iteration: 5 ====================
[ 0.14099844  0.0815044  -0.04003461 -0.02621942 -0.03773969  0.00130672
 -0.02406035 -0.11305888  0.01259053]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.634
||  s  || FPR. || FNR. ||
||  0  || 0.68 || 0.07 ||
||  1  || 0.88 || 0.03 ||


Converged
================= iteration: 6 ====================
[ 0.14054583  0.07886258 -0.04115345 -0.02611605 -0.03496266  0.00330819
 -0.02811377 -0.11261971  0.02132493]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.632
||  s  || FPR. || FNR. ||
||  0  || 0.68 || 0.07 ||
||  1  || 0.88 || 0.04 ||


Converged
=======



Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.84 || 0.04 ||


Converged
================= iteration: 12 ====================
[ 0.14611018  0.07693394 -0.03884425 -0.02613206 -0.03706652  0.0144962
 -0.02411795 -0.11206492  0.0224511 ]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.84 || 0.04 ||


Converged
================= iteration: 13 ====================
[ 0.14568085  0.07950917 -0.03857017 -0.02684625 -0.0407464   0.00849548
 -0.02166563 -0.10961091  0.02502087]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.613
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.84 || 0.04 ||


Converged
================= iteration: 14 ====================
[ 0.14559472  0.07950916 -0.03857017 -0.02684625 -0.0407464   0.00849548
 -0.02166563 -0.10961091  0.02502087]
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.613
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.84 || 0.04 ||


Converged
=====

### Saving the obtained weight vectors

In [7]:
scipy.io.savemat("../Result_Data/DworkHeuristic/W_matrix_heuristic",{'dummykey':weight_matrix})
scipy.io.savemat("../Result_Data/DworkHeuristic/no_constraints_heuristic",{'dummykey':num_cons_array})

## Here we evaluate different metrics for all the weight vectors we got from the training process above

Note that we have to use the same validation splits to ensure testing is really done on validation data and not on data already seen during training.

In [9]:
# for computing atkinson's index and social welfare
alpha = 0.8

delta_array = [0]
num_vio_array1 = np.zeros((iterations,len(delta_array)))
num_vio_array2 = np.zeros((iterations,len(delta_array)))
num_vio_array3 = np.zeros((iterations,len(delta_array)))

avg_vio_array1 = np.zeros(iterations)
avg_vio_array2 = np.zeros(iterations)
avg_vio_array3 = np.zeros(iterations)

#false positive rate
fpr_matrix = np.zeros(iterations)
# false negative rate
fnr_matrix = np.zeros(iterations)
# holds the absolute difference in percentage of positive outcomes
dp_matrix = np.zeros(iterations)
# holds the atkinsons index
at_matrix = np.zeros(iterations)
# holds ge2 unfairness
ge_matrix = np.zeros(iterations)
# holds accuracy
accuracy_matrix = np.zeros(iterations)
# holds social welfare
welfare_matrix = np.zeros(iterations)


i = 0
for train_index, test_index in kf.split(X):
    print("#############cross validation iteration " + str(i) + " #########################")
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_control_train = {'race':x_control['race'][train_index]}
    x_control_test = {'race':x_control['race'][test_index]}
    num_points, num_features = x_train.shape

    
    shuffle(vio_list_uncons)
    for k in range(iterations):
        print("================= iteration: " + str(k) + " ====================")
        
        w_cons = weight_matrix[k,:]
        
        
        train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_cons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)
        y_pred_test = np.sign(np.dot(x_test, w_cons))
        y_pred_test2 = np.reciprocal(1+np.exp(-np.dot(x_test, w_cons)))
        y_pred_test3 = (np.dot(x_test, w_cons))/5
        

        group_array = x_control_test['race']

        # total number of positive and negative points
        unique, counts = np.unique(y_test, return_counts=True)
        n_neg1 = 0
        n_pos1 = 0

        n_neg0 = 0
        n_pos0 = 0


        # total number of data points
        n = y_test.shape[0]
        unique, counts = np.unique(group_array, return_counts=True)
        # number of people in group 0
        n0 = counts[0]
        # number of people in group 1
        n1 = counts[1]

        # positive prediction count for both classes and for predicted labels
        pos_count0_pred = 0
        pos_count1_pred = 0
        fp_count0 = 0
        fp_count1 = 0
        fn_count0 = 0
        fn_count1 = 0

        counter = 0
        for group_membership in group_array:
            if group_membership == 1:
                if y_test[counter] == 1:
                    n_pos1 += 1
                if y_test[counter] == -1:
                    n_neg1 += 1
                # demographic parity check
                if y_pred_test[counter] == 1:
                    pos_count1_pred += 1
                #fpr fnr check
                if y_test[counter] == 1 and y_pred_test[counter] == -1:
                    fn_count1 += 1
                if y_test[counter] == -1 and y_pred_test[counter] == 1:
                    fp_count1 += 1
            elif group_membership == 0:
                if y_test[counter] == 1:
                    n_pos0 += 1
                if y_test[counter] == -1:
                    n_neg0 += 1
                # demographic parity check
                if y_pred_test[counter] == 1:
                    pos_count0_pred += 1
                #fpr fnr check
                if y_test[counter] == 1 and y_pred_test[counter] == -1:
                    fn_count0 += 1
                if y_test[counter] == -1 and y_pred_test[counter] == 1:
                    fp_count0 += 1
            else:
                print("ERROR")
            counter += 1
        
        ge_matrix[k] += ge2(y_pred_test, y_test) / cross_val_iter
        accuracy_matrix[k] += test_score / cross_val_iter
        fpr_matrix[k] += (fp_count0/n_neg0 - fp_count1/n_neg1) / cross_val_iter
        fnr_matrix[k] += (fn_count0/n_pos0 - fn_count1/n_pos1) / cross_val_iter
        dp_matrix[k] += (pos_count0_pred/n0 - pos_count1_pred/n1) / cross_val_iter
        at_matrix[k] += atk(y_pred_test, y_test, alpha) / cross_val_iter
        welfare_matrix[k] += eval_util(y_pred_test, y_test, alpha)/ cross_val_iter


        
        print("evaluating avg violation")
        avg_vio_array1[k] += avg_Dwork_violation(y_pred_test, x_test, 0, d_max)/cross_val_iter
        avg_vio_array2[k] += avg_Dwork_violation(y_pred_test2, x_test, 0, d_max)/cross_val_iter
        avg_vio_array3[k] += avg_Dwork_violation(y_pred_test3, x_test, 0, d_max)/cross_val_iter
        for j in range(len(delta_array)):
            print("evaluating no violation")
            d = delta_array[j]
            num_vio_array1[k][j] += num_Dwork_violation(y_pred_test, x_test, d, d_max)/cross_val_iter
            num_vio_array2[k][j] += num_Dwork_violation(y_pred_test2, x_test, d, d_max)/cross_val_iter
            num_vio_array3[k][j] += num_Dwork_violation(y_pred_test3, x_test, d, d_max)/cross_val_iter
    
    i += 1
dp_matrix = np.absolute(dp_matrix)
fpr_matrix = np.absolute(fpr_matrix)
fnr_matrix = np.absolute(fnr_matrix)

#############cross validation iteration 0 #########################
================= iteration: 0 ====================
[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.686
||  s  || FPR. || FNR. ||
||  0  || 0.27 || 0.34 ||
||  1  || 0.68 || 0.10 ||


evaluating avg violation
evaluating no violation
================= iteration: 1 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.623
||  s  || FPR. || FNR. ||
||  0  || 0.63 || 0.12 ||
||  1  || 0.84 || 0.04 ||


evaluating avg violation
evaluating no violation
================= iteration: 2 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.621
||  s  || FPR. || FNR. ||
||  0  || 0.63 || 0.13 ||
||  1  || 0.84 || 0.04 ||


evaluating avg violation
evaluating no violation
================= iteration: 3 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.621
||  s  || FPR. || FNR. ||
||  0  || 0.63 || 0.13 ||
||  1  || 0.84 || 0.04 ||


evaluating avg violation
evaluating no violation
================= iteration: 4 ===


Accuracy: 0.631
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.89 || 0.03 ||


evaluating avg violation
evaluating no violation
================= iteration: 17 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.629
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.89 || 0.03 ||


evaluating avg violation
evaluating no violation
================= iteration: 18 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.629
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.89 || 0.03 ||


evaluating avg violation
evaluating no violation
================= iteration: 19 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.629
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.89 || 0.03 ||


evaluating avg violation
evaluating no violation
#############cross validation iteration 2 #########################
================= iteration: 0 ====================
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.648
||  s  || FPR.

||  0  || 0.70 || 0.07 ||
||  1  || 0.89 || 0.06 ||


evaluating avg violation
evaluating no violation
================= iteration: 13 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.636
||  s  || FPR. || FNR. ||
||  0  || 0.70 || 0.07 ||
||  1  || 0.89 || 0.06 ||


evaluating avg violation
evaluating no violation
================= iteration: 14 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.636
||  s  || FPR. || FNR. ||
||  0  || 0.70 || 0.07 ||
||  1  || 0.89 || 0.06 ||


evaluating avg violation
evaluating no violation
================= iteration: 15 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.636
||  s  || FPR. || FNR. ||
||  0  || 0.70 || 0.07 ||
||  1  || 0.89 || 0.06 ||


evaluating avg violation
evaluating no violation
================= iteration: 16 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.638
||  s  || FPR. || FNR. ||
||  0  || 0.70 || 0.06 ||
||  1  || 0.89 || 0.06 ||


evaluating avg violation
evaluating no violation
=

evaluating no violation
================= iteration: 9 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.591
||  s  || FPR. || FNR. ||
||  0  || 0.68 || 0.07 ||
||  1  || 0.91 || 0.03 ||


evaluating avg violation
evaluating no violation
================= iteration: 10 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.591
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.08 ||
||  1  || 0.91 || 0.03 ||


evaluating avg violation
evaluating no violation
================= iteration: 11 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.591
||  s  || FPR. || FNR. ||
||  0  || 0.68 || 0.07 ||
||  1  || 0.91 || 0.03 ||


evaluating avg violation
evaluating no violation
================= iteration: 12 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.591
||  s  || FPR. || FNR. ||
||  0  || 0.68 || 0.07 ||
||  1  || 0.91 || 0.03 ||


evaluating avg violation
evaluating no violation
================= iteration: 13 ====================
[1. 1. 1. ... 1. 1. 1.]


Ac

================= iteration: 5 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.616
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.84 || 0.06 ||


evaluating avg violation
evaluating no violation
================= iteration: 6 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.84 || 0.06 ||


evaluating avg violation
evaluating no violation
================= iteration: 7 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.84 || 0.06 ||


evaluating avg violation
evaluating no violation
================= iteration: 8 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.617
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.09 ||
||  1  || 0.84 || 0.06 ||


evaluating avg violation
evaluating no violation
================= iteration: 9 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.617
||  s  || FPR.

================= iteration: 1 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.645
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.09 ||
||  1  || 0.80 || 0.04 ||


evaluating avg violation
evaluating no violation
================= iteration: 2 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.643
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.08 ||
||  1  || 0.81 || 0.04 ||


evaluating avg violation
evaluating no violation
================= iteration: 3 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.643
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.08 ||
||  1  || 0.81 || 0.04 ||


evaluating avg violation
evaluating no violation
================= iteration: 4 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.643
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.08 ||
||  1  || 0.81 || 0.04 ||


evaluating avg violation
evaluating no violation
================= iteration: 5 ====================
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.643
||  s  || FPR.

### Saving the obtained results

In [10]:
dwork_avg = np.column_stack((avg_vio_array1,avg_vio_array2,avg_vio_array3))
dwork_no = np.column_stack((num_vio_array1[:,0],num_vio_array2[:,0],num_vio_array3[:,0]))

scipy.io.savemat("../Result_Data/DworkHeuristic/dwork_avg_matrix_heuristic",{'dummykey':dwork_avg})
scipy.io.savemat("../Result_Data/DworkHeuristic/dwork_no_matrix_heuristic",{'dummykey':dwork_no})
scipy.io.savemat("../Result_Data/DworkHeuristic/fpr_matrix_heuristic",{'dummykey':fpr_matrix})
scipy.io.savemat("../Result_Data/DworkHeuristic/fnr_matrix_heuristic",{'dummykey':fnr_matrix})
scipy.io.savemat("../Result_Data/DworkHeuristic/dp_matrix_heuristic",{'dummykey':dp_matrix})
scipy.io.savemat("../Result_Data/DworkHeuristic/ge_matrix_heuristic",{'dummykey':ge_matrix})
scipy.io.savemat("../Result_Data/DworkHeuristic/accuracy_matrix_heuristic",{'dummykey':accuracy_matrix})
scipy.io.savemat("../Result_Data/DworkHeuristic/at_matrix_heuristic",{'dummykey':at_matrix})
scipy.io.savemat("../Result_Data/DworkHeuristic/welfare_matrix_heuristic",{'dummykey':welfare_matrix})